# **ZM → VALLE DE MÉXICO**



## **LIBRERÍAS A UTILIZAR**

In [ ]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
!pip install mapclassify

In [ ]:
!pip install libpysal

In [ ]:
!pip install fiona
!pip install rtree
!pip install pygeos

In [ ]:
import fiona
import rtree
import pygeos

In [ ]:
!pip install geopandas

In [ ]:
!pip install matplotlib

In [ ]:
import numpy as np
import mapclassify
import pandas as pd
import libpysal as ps
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
!pip install contextily

In [ ]:
import matplotlib.patches as mpatches
import contextily as ctx

## **BD: AGEBs SHAPE, INEGI 2020**

In [ ]:
AGEBS= gpd.read_file('AgebsUrbana_ValleMex_ZM.shp')
print ('Forma de los datos:',AGEBS.shape)
AGEBS.head(5)

In [ ]:
AGEBS.crsAGEBS.crs

## **BD: ÁREAS VERDES, INEGI 2020**

In [ ]:
Area_Verde= gpd.read_file('SIA_ValleMex_ZM.shp')
Area_Verde.head()

In [ ]:
Area_Verde.crs

## **BD: MEDICIÓN DE POBREZA, CONEVAL 2015**

In [ ]:
Pobreza_ageb = pd.read_csv ('Pobreza_ValleMX_csv')
Pobreza_ageb.head(5)

## **HOMOLOGAR BASES DE DATOS**

In [ ]:
AGEBS = AGEBS.rename({'CVEGEO':'Clave_AGEB'}, axis=1)
Pobreza_ageb = Pobreza_ageb.rename({'cvegeo':'Clave_AGEB'}, axis=1)
Pobreza_ageb = Pobreza_ageb.rename({'pobreza':'Rango_Pobreza'}, axis=1)
Pobreza_ageb = Pobreza_ageb.rename({'pobreza_ext':'Rango_PExtrema'}, axis=1)

In [ ]:
Pobreza_ageb=Pobreza_ageb.dropna()

In [ ]:
Pobreza_ageb['Min_RP'] = Pobreza_ageb.Rango_Pobreza.str.extract('(\d+)').astype(int)
Pobreza_ageb['Max_RP'] = Pobreza_ageb.Rango_Pobreza.str.extract('( \d+)').astype(int)
Pobreza_ageb['Pobre_AV'] = ((Pobreza_ageb['Min_RP'] + Pobreza_ageb['Max_RP']) / 2)
Pobreza_ageb2 = Pobreza_ageb[['Clave_AGEB','nom_mun', 'Rango_Pobreza', 'Pobre_AV' ]]

In [ ]:
Pobreza_ageb2.set_index('Clave_AGEB')
AGEBS.set_index('Clave_AGEB')

In [ ]:
Pobreza_AGEB = AGEBS.merge(Pobreza_ageb2 ,left_on='Clave_AGEB', right_on='Clave_AGEB',how='left')
Pobreza_A = Pobreza_AGEB.copy()
print ('Forma de los datos:',Pobreza_AGEB.shape)
Pobreza_AGEB.head()

In [ ]:
Pobreza_A = Pobreza_A.fillna(0)
Pobreza_A.head()

In [ ]:
Pobreza_AGEB = Pobreza_AGEB.dropna()

In [ ]:
Pobreza_A.crs

In [ ]:
Pobreza_A=Pobreza_A.to_crs(epsg=3857)

In [ ]:
mapclassify.NaturalBreaks(Pobreza_A['Pobre_AV'].values, k=5)

## **MAPA DE POBREZA POR AGEBs**

In [ ]:
fig, ax = plt.subplots(1,figsize=(16, 18)) 
Base = Pobreza_A.plot(column='Pobre_AV', 
              alpha=1, 
              scheme='NaturalBreaks', 
              legend=True, 
              cmap='YlOrRd',
              classification_kwds={'k':5},
              figsize=(8,8),
              ax=ax)
plt.title("Rango de Pobreza en la ZM del Valle de México, 2015 (%)", size = 25)
ax.set_axis_off() 
ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite, alpha=0.3) 
plt.show()

In [ ]:
Pobreza_AGEB=Pobreza_AGEB.to_crs(epsg=3857)

## **MAPA DE POBREZA EXTREMA POR AGEBs**

In [ ]:
fig, ax = plt.subplots(1,figsize=(16, 18)) 
Base = Pobreza_AGEB.plot(column='Pobre_AV', 
              alpha=1, 
              scheme='NaturalBreaks', 
              legend=True, 
              cmap='YlOrRd',
              classification_kwds={'k':5},
              figsize=(8,8),
              ax=ax)
plt.title("Rango de Pobreza en la ZM del Valle de México, 2015 (%)", size = 25)
ax.set_axis_off() 
ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite, alpha=0.3) 
plt.show()

## **ÁREAS VERDES → OPEN STREET MAPS**

In [ ]:
!pip install osmnx
import osmnx as ox

In [ ]:
!pip install git+git://github.com/geopandas/geopandas.git

In [ ]:
tags = {'leisure': True}
AreaV = ox.geometries_from_place('Ciudad de México', tags)
AreaV.head(10)

In [ ]:
tags = {'leisure': True}
AreaV1 = ox.geometries_from_place('Estado de México', tags)
AreaV1.head(10)

In [ ]:
tags = {'leisure': True}
AreaV2 = ox.geometries_from_place('Hidalgo', tags)
AreaV2.head(10)

## **SOLO PARQUES → OPEN STREET MAPS**

In [ ]:
Parque= AreaV.drop(AreaV[AreaV['leisure']!='park'].index)
Parque1= AreaV1.drop(AreaV1[AreaV1['leisure']!='park'].index)
Parque2= AreaV2.drop(AreaV2[AreaV2['leisure']!='park'].index)

In [ ]:
Parque.head()

In [ ]:
Parque.crs

In [ ]:
frames = [Parque, Parque1, Parque2]

In [ ]:
Parques=pd.concat(frames)

In [ ]:
Parques.shape

In [ ]:
Parques=Parques.to_crs(epsg=6372)
Parques.head()
AGEBS=AGEBS.to_crs(epsg=6372)

## **PLOTEO PARQUES → INEGI**

In [ ]:
colors = ['#A1E2E6', '#E6BDA1', '#B3A16B', '#678072', '#524A4A']

In [ ]:
Parque_Inegi = gpd.clip(Area_Verde, AGEBS)
Parque_Inegi.head()

In [ ]:
Parques_Leon= AGEBS.plot(color= colors[2], alpha=.2, figsize=(16, 18))
Parque_Inegi.plot(ax=Parques_Leon, color= '#039e7d',linewidth= 1.5)
plt.title("ZM del Valle de México\n Áreas Verdes de INEGI", fontsize=25, color= '#03989e')
plt.axis('off')
plt.show()

In [ ]:
Parque_Clip = gpd.clip(Parques, AGEBS)
Parque_Clip.head()

## **PLOTEO PARQUES → OPEN STREET MAPS**

In [ ]:
Parque_Clips = gpd.clip(Area_Verde, AGEBS)
Parque_Clips.head()

In [ ]:
Parques_LEON_OSM2= AGEBS.plot(color=colors[2], alpha=0.2, figsize=(16, 18))
Parque_Clip.plot(ax=Parques_LEON_OSM2, color='#039e7d')
plt.title("ZM del Valle de México\n Áreas Verdes de Open Street Map", fontsize=25, color= '#03989e')
plt.axis('off')
plt.show()

In [ ]:
AreaV.crs

In [ ]:
AreaV_INEGI=Parque_Clips.to_crs(epsg=3857)
AreaV_OSM=Parque_Clip.to_crs(epsg=3857)

## **PLOTEO → INEGI & POBREZA**

In [ ]:
fig, ax = plt.subplots(1,figsize=(16, 18)) 
Base = Pobreza_A.plot(column='Pobre_AV',  
              alpha=1, 
              scheme='NaturalBreaks', 
              legend=True, 
              cmap='YlOrRd',
              classification_kwds={'k':5},
              figsize=(8,8),
              ax=ax)
AreaV_INEGI.plot(ax=Base, color='#039e7d')
plt.title("Rango de Pobreza en el Valle de México, 2015 (%) \n Áreas Verdes de INEGI", fontsize=25, color = '#03989e')
ax.set_axis_off() 
ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite, alpha=0.3)
plt.show()

## **PLOTEO → OSM & POBREZA**

In [ ]:
fig, ax = plt.subplots(1,figsize=(16, 18)) 
Base = Pobreza_A.plot(column='Pobre_AV',  
              alpha=1, 
              scheme='NaturalBreaks', 
              legend=True, 
              cmap='YlOrRd',
              classification_kwds={'k':5},
              figsize=(8,8),
              ax=ax)
AreaV_OSM.plot(ax=Base, color='#039e7d')
plt.title("Rango de Pobreza en el Valle de México, 2015 (%) \n Áreas Verdes de Open Street Map", fontsize=25, color = '#03989e')
ax.set_axis_off() 
ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite, alpha=0.3)
plt.show()